In [ ]:
# Gradio interface prototype
from typing import List
import gradio as gr
#from aisurveywriter import generate_paper_survey

def generate_paper_survey(msg, paths, sp):
    print(f"INSIDE GEN PAPER SURVEY: {msg}, {paths}, {sp}")
    return "TESTING"

def chat_fn(message, history, refs, save_path):
    print(f"DEBUG: message={message}, HISTORY={history}, REFS={refs}, PDF_PATHS={save_path}")
    pdf_paths = [file.name for file in refs]
    result = generate_paper_survey(message, pdf_paths, save_path)
    return "The genereated papar content is: " + result

demo = gr.ChatInterface(
    fn=chat_fn,
    textbox=gr.Textbox(placeholder="Enter the subject of the survey paper..."),
    additional_inputs=[
        gr.File(label="Upload reference PDFs", file_types=[".pdf"], file_count="multiple"),
        gr.Textbox(label="Save path and name", placeholder="Enter the full path to save the paper (including its filename)"),
    ],
    title="Survey Paper Writer",
    description="Provide a subject, reference PDFs, and a save path to generate and save a survey paper.",
)
demo.launch()

In [ ]:
# bibtex from (title and author) or DOI

import requests
import bibtexparser
from bibtexparser.bparser import BibTexParser
from bibtexparser.bibdatabase import BibDatabase

def search_crossref(title, author):
    """
    Search CrossRef API using title and author to retrieve the DOI.
    """
    url = "https://api.crossref.org/works"
    params = {"query.title": title, "query.author": author, "rows": 1}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        items = data.get("message", {}).get("items", [])
        if items:
            return items[0].get("DOI")
    return None

def get_bibtext(doi, cache={}):
    """
    Use DOI Content Negotiation to retrieve a string with the bibtex entry.
    """
    if doi in cache:
        return cache[doi]
    url = f'https://doi.org/{doi}'
    headers = {'Accept': 'application/x-bibtex'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        bibtext = response.text
        cache[doi] = bibtext
        return bibtext
    return None

def get_abstract(doi):
    """
    Retrieve the abstract of a paper using the CrossRef API.
    """
    url = f'https://api.crossref.org/works/{doi}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        print(data.get("message",{}))
        return data.get('message', {}).get('abstract', None)
    return None

def get_bibtex_entry(title, author, bibtext_cache={}):
    """
    Retrieve BibTeX entry using title and author.
    """
    doi = search_crossref(title, author)
    if not doi:
        print("DOI not found for given title and author.")
        return None
    
    bibtext = get_bibtext(doi, cache=bibtext_cache)
    if not bibtext:
        print("BibTeX entry not found.")
        return None
    
    parser = BibTexParser()
    parser.ignore_nonstandard_types = False
    bibdb = bibtexparser.loads(bibtext, parser)
    entry, = bibdb.entries
    entry['link'] = f'https://doi.org/{doi}'
    
    if 'author' in entry:
        entry['author'] = ' and '.join(entry['author'].rstrip(';').split('; '))
    
    entry['ID'] = doi.split('/')[-1]
    
    # Retrieve and add abstract
    abstract = get_abstract(doi)
    if abstract:
        entry['abstract'] = abstract
    
    return entry

def entries_to_str(entries):
    """
    Pass a list of bibtexparser entries and return a bibtex formatted string.
    """
    db = BibDatabase()
    db.entries = entries
    return bibtexparser.dumps(db)

# Example usage
title = "Chemistry of Materials Nanoarchitectonics for Two-Dimensional Films"
author = "Ariga K"
entry = get_bibtex_entry(title, author)
if entry:
    print(entries_to_str([entry]))

print(entry["doi"])

In [ ]:
from aisurveywriter import generate_paper_survey
from aisurveywriter.utils import get_all_files_from_paths

generate_paper_survey(
    subject="Langmuir Monolayers and Langmuir-Blodgett Films",
    ref_paths=get_all_files_from_paths("D:/Dev/aisurveywriter/refexamples"),
    save_path="here.tex",
)

In [1]:
from aisurveywriter.tasks import ReferenceExtractor
from aisurveywriter.core.llm_handler import LLMHandler
import aisurveywriter.core.file_handler as fh

import os
os.environ["GOOGLE_API_KEY"]=""

prompt = fh.read_yaml("../templates/prompt_config.yaml")["reference_extract_prompt"]
ref = ReferenceExtractor(llm=LLMHandler(model="deepseek-r1:7b", model_type="ollama"),
                         ref_paths=["../refexamples/ArigaK2023_ChemOfMat.pdf"], prompt=prompt,
                         raw_save_path="./raw.ref", rawbib_save_path="./rawbib.bib",
                         bib_save_path="./final.bib")
ref()

/home/juliocesar/miniconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(ReferenceExtractor) ==> started extracting references from pdf ArigaK2023_ChemOfMat.pdf


ResponseError: model "deepseek-r1:8b" not found, try pulling it first (status code: 404)

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_abstract_from_doi(doi):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "application/vnd.citationstyles.csl+json",
    }
    
    # Step 1: Use CrossRef API
    try:
        crossref_url = f"https://api.crossref.org/works?filter=has-abstract:true/{doi}"
        response = requests.get(crossref_url, headers=headers)
        if response.status_code == 200:
            print("yes")
            data = response.json()
            abstract = data["message"].get("abstract")
            if abstract:
                return abstract
    except Exception as e:
        print(f"CrossRef failed: {e}")

    # Step 2: Access DOI redirection URL
    try:
        doi_url = f"https://doi.org/{doi}"
        response = requests.get(doi_url, headers=headers, allow_redirects=True)
        print(response)
        if response.status_code == 200:
            print(response.text)
            print("abstract" in response.text)
            return None
    except Exception as e:
        print(f"DOI redirection failed: {e}")

    # Step 3: If all else fails, return None
    return None

# Example usage
doi = "10.1021/acs.chemrev.1c00754"
abstract = get_abstract_from_doi(doi)
if abstract:
    print("Abstract found:", abstract)
else:
    print("Abstract not found.")
